## Creating Text-Fabric dataset (from LowFat XML trees)

<pre>
    Code version: 0.7 (February 17, 2024)
    Data version: February 10, 2024 (<a href="https://github.com/tonyjurg/Nestle1904LFT/tree/main/resources/xml/20240210#readme" target="_blank">Readme</a>)
</pre>

## Table of content <a class="anchor" id="TOC"></a>
* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Read LowFat XML data and store in pickle</a>
    * <a href="#bullet2x1">2.1 - Required libraries</a>
    * <a href="#bullet2x2">2.2 - Import various libraries</a>
    * <a href="#bullet2x3">2.3 - Initialize global data</a>
    * <a href="#bullet2x4">2.4 - Add parent info to each node of the XML tree</a>
    * <a href="#bullet2x5">2.5 - Process the XML data and store dataframe in pickle</a>
* <a href="#bullet3">3 - Production Text-Fabric from pickle input</a>
    * <a href="#bullet3x1">3.1 - Load libraries and initialize some data</a>
    * <a href="#bullet3x2">3.2 - Optionaly export to Excel for investigation</a>
    * <a href="#bullet3x3">3.3 - Running the TF walker function</a>
* <a href="#bullet4">4 - Publish product on github</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to TOC](#TOC)

The source data for the conversion are the LowFat XML trees files representing the macula-greek version of the Nestle 1904 Greek New Testment (British Foreign Bible Society, 1904). The starting dataset is formatted according to Syntax diagram markup by the Global Bible Initiative (GBI). The most recent source data can be found on github https://github.com/Clear-Bible/macula-greek/tree/main/Nestle1904/lowfat. 

Attribution: "MACULA Greek Linguistic Datasets, available at https://github.com/Clear-Bible/macula-greek/". 

The production of the Text-Fabric files consist of two phases. First one is the creation of piclke files (<a href="#bullet2">section 2</a>). The second phase is the the actual Text-Fabric creation process (<a href="#bullet3">section 3</a>). The process can be depicted as follows:

<img src='../images/highlevelconversion.png'>

# 2 - Read LowFat XML data and store in pickle <a class="anchor" id="bullet2"></a>
##### [Back to TOC](#TOC)

This script harvests all information from the LowFat tree data (XML nodes), puts it into a Panda DataFrame and stores the result per book in a pickle file. Note: pickling (in Python) is serialising an object into a disk file (or buffer). See also the [Python3 documentation](https://docs.python.org/3/library/pickle.html).

Within the context of this script, the term 'Leaf' refers to nodes that contain the Greek word as data. These nodes are also referred to as 'terminal nodes' since they do not have any children, similar to leaves on a tree. Additionally, Parent1 represents the parent of the leaf, Parent2 represents the parent of Parent1, and so on. For a visual representation, please refer to the following diagram.

<img src="../images/leaves_and_root.png">

For a full description of the source data see document [MACULA Greek Treebank for the Nestle 1904 Greek New Testament.pdf](https://github.com/Clear-Bible/macula-greek/blob/main/doc/MACULA%20Greek%20Treebank%20for%20the%20Nestle%201904%20Greek%20New%20Testament.pdf)

## 2.1 Required libraries<a class="anchor" id="bullet2x1"></a>
##### [Back to TOC](#TOC)

The scripts in this notebook require (beside text-fabric) the following Python libraries to be installed in the environment:

<pre>
   pandas
   openpyxl
</pre>

You can install any missing library from within Jupyter Notebook using either `pip` or `pip3`. (eg.: !pip3 install pandas)

## 2.2 - Import various libraries<a class="anchor" id="bullet2x2"></a>
##### [Back to TOC](#TOC)

In [3]:
import pandas as pd
import sys
import os
import time
import pickle

import re  #regular expressions
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET


## 2.3 - Initialize global data<a class="anchor" id="bullet2x3"></a>
##### [Back to TOC](#TOC)

The following global data initializes the script, gathering the XML data to store it into the pickle files.

IMPORTANT: To ensure proper creation of the Text-Fabric files on your system, it is crucial to adjust the values of BaseDir, InputDir, and OutputDir to match the location of the data and the operating system you are using. In this Jupyter Notebook, Windows is the operating system employed.

In [4]:
BaseDir = 'D:\\TF\\'
XmlDir = BaseDir+'xml\\'
PklDir = BaseDir+'pkl\\'
XlsxDir = BaseDir+'xlsx\\'
# note: create output directory prior running this part

#         key: filename,       [0]=book_long,   [1]=book_num,  [3]=book_short
bo2book = {'01-matthew':       ['Matthew',         '1',        'Matt'],
           '02-mark':          ['Mark',            '2',        'Mark'],
           '03-luke':          ['Luke',            '3',        'Luke'],
           '04-john':          ['John',            '4',        'John'],
           '05-acts':          ['Acts',            '5',        'Acts'],
           '06-romans':        ['Romans',          '6',        'Rom'],
           '07-1corinthians':  ['I_Corinthians',   '7',        '1Cor'],
           '08-2corinthians':  ['II_Corinthians',  '8',        '2Cor'],
           '09-galatians':     ['Galatians',       '9',        'Gal'],
           '10-ephesians':     ['Ephesians',       '10',       'Eph'],
           '11-philippians':   ['Philippians',     '11',       'Phil'],
           '12-colossians':    ['Colossians',      '12',       'Col'],
           '13-1thessalonians':['I_Thessalonians', '13',       '1Thess'],
           '14-2thessalonians':['II_Thessalonians','14',       '2Thess'],
           '15-1timothy':      ['I_Timothy',       '15',       '1Tim'],
           '16-2timothy':      ['II_Timothy',      '16',       '2Tim'],
           '17-titus':         ['Titus',           '17',       'Titus'],
           '18-philemon':      ['Philemon',        '18',       'Phlm'],
           '19-hebrews':       ['Hebrews',         '19',       'Heb'],
           '20-james':         ['James',           '20',       'Jas'],
           '21-1peter':        ['I_Peter',         '21',       '1Pet'],
           '22-2peter':        ['II_Peter',        '22',       '2Pet'],
           '23-1john':         ['I_John',          '23',       '1John'],
           '24-2john':         ['II_John',         '24',       '2John'],
           '25-3john':         ['III_John',        '25',       '3John'],     
           '26-jude':          ['Jude',            '26',       'Jude'],
           '27-revelation':    ['Revelation',      '27',       'Rev']}
#bo2book = {           '18-philemon':      ['Philemon',        '18',       'Phlm']}


## 2.4 - Add parent info to each node of the XML tree<a class="anchor" id="bullet2x4"></a>
##### [Back to TOC](#TOC)

In order to be able to traverse from the 'leafs' upto the root of the tree, it is required to add information to each node pointing to the parent of each node. The terminating nodes of an XML tree are called "leaf nodes" or "leaves." These nodes do not have any child elements and are located at the end of a branch in the XML tree. Leaf nodes contain the actual data or content within an XML document. In contrast, non-leaf nodes are called "internal nodes," which have one or more child elements.

In [49]:
def build_parent_map(tree):
    parent_map = {c:p for p in tree.iter() for c in p}
    return parent_map

def getParent(et, parent_map):
    return parent_map.get(et)

# Usage to build the map:
#  tree = ET.parse(InputFile)
#  parent_map = build_parent_map(tree)
# Then, whenever you need a parent of an element:
# parent = getParent(some_element, parent_map)


## 2.5 - Process the XML data and store dataframe in pickle<a class="anchor" id="bullet2x5"></a>
##### [Back to TOC](#TOC)

This code processes books in the correct order. Firstly, it parses the XML and adds parent information to each node. Then, it loops through the nodes and checks if it is a 'leaf' node, meaning it contains only one word. If it is a 'leaf' node, the following steps are performed:

* Loads the xml files into memmory and build a parent-child mapping.
* Traverses from the 'leaf' node up to the root and adds information from the parent, grandparent, and so on, to the 'leaf' node.
* Once it reaches the root, it stops and stores all the gathered information in a dataframe that will be added to the full_dataframe.
* After processing all the nodes for a specific book, the full_dataframe is exported to a pickle file specific to that book.

Note that this script takes a long time to execute (due to the large number of itterations). However, once the XML data is converted to PKL, there is no need to rerun (unless the source XML data is updated).

In [50]:
# Create the pickle files

import os
import time
import pandas as pd
from lxml import etree as ET
import re
import pickle

# Set global variables
WordOrder = 1
CollectedItems = 0

def build_parent_map(tree):
    return {c: p for p in tree.iter() for c in p}

def getParent(et, parent_map):
    return parent_map.get(et)

def process_element(elem, book_info, WordOrder, parent_map):
    global CollectedItems
    Leafref = re.sub(r'[!: ]', " ", elem.attrib.get('ref')).split()
    elem_attrib = dict(elem.attrib)  # Create a copy of the attributes using dict()

    # Adding new or modifying existing attributes
    elem_attrib.update({
        'word_order': WordOrder,
        'LeafName': elem.tag,
        'word': elem.text,
        'book_long': book_info[0],
        'booknum': int(book_info[1]),
        'book_short': book_info[2],
        'chapter': int(Leafref[1]),
        'verse': int(Leafref[2]),
        'parents': 0  # Initialize 'parents' attribute
    })

    parentnode = getParent(elem, parent_map)
    index = 0
    while parentnode is not None:
        index += 1
        parent_attribs = {
            f'Parent{index}Name': parentnode.tag,
            f'Parent{index}Type': parentnode.attrib.get('type'),
            f'Parent{index}Appos': parentnode.attrib.get('appositioncontainer'),
            f'Parent{index}Class': parentnode.attrib.get('class'),
            f'Parent{index}Rule': parentnode.attrib.get('rule'),
            f'Parent{index}Role': parentnode.attrib.get('role'),
            f'Parent{index}Cltype': parentnode.attrib.get('cltype'),
            f'Parent{index}Unit': parentnode.attrib.get('unit'),
            f'Parent{index}Junction': parentnode.attrib.get('junction'),
            f'Parent{index}SN': parentnode.attrib.get('SN'),
            f'Parent{index}WGN': parentnode.attrib.get('WGN')
        }
        elem_attrib.update(parent_attribs)
        parentnode = getParent(parentnode, parent_map)

    elem_attrib['parents'] = index

    CollectedItems += 1
    return elem_attrib, WordOrder + 1

# Process books
for bo, book_info in bo2book.items():
    CollectedItems = 0
    SentenceNumber = 0
    WordGroupNumber = 0
    data_list = []  # List to store data dictionaries

    InputFile = os.path.join(XmlDir, f'{bo}.xml')
    OutputFile = os.path.join(PklDir, f'{bo}.pkl')
    print(f'Processing {book_info[0]} at {InputFile}')

    try:
        tree = ET.parse(InputFile)
        parent_map = build_parent_map(tree)
    except Exception as e:
        print(f"Error parsing XML file {InputFile}: {e}")
        continue

    start_time = time.time()

    for elem in tree.iter():
        if elem.tag == 'sentence':
            SentenceNumber += 1
            elem.set('SN', str(SentenceNumber))
        elif elem.tag == 'error':
            elem.tag = 'wg'
        if elem.tag == 'wg':
            WordGroupNumber += 1
            elem.set('WGN', str(WordGroupNumber))
        if elem.tag == 'w':
            elem_attrib, WordOrder = process_element(elem, book_info, WordOrder, parent_map)
            data_list.append(elem_attrib)

    full_df = pd.DataFrame(data_list)  # Create DataFrame once after processing all elements
    with open(OutputFile, 'wb') as output:
        pickle.dump(full_df, output)

    print(f"\nFound {CollectedItems} items in {time.time() - start_time:.2f} seconds\n")

Processing Matthew at D:\TF\xml\01-matthew.xml

Found 18299 items in 1.61 seconds

Processing Mark at D:\TF\xml\02-mark.xml

Found 11277 items in 1.04 seconds

Processing Luke at D:\TF\xml\03-luke.xml

Found 19456 items in 4.98 seconds

Processing John at D:\TF\xml\04-john.xml

Found 15643 items in 1.33 seconds

Processing Acts at D:\TF\xml\05-acts.xml

Found 18393 items in 1.80 seconds

Processing Romans at D:\TF\xml\06-romans.xml

Found 7100 items in 0.77 seconds

Processing I_Corinthians at D:\TF\xml\07-1corinthians.xml

Found 6820 items in 0.59 seconds

Processing II_Corinthians at D:\TF\xml\08-2corinthians.xml

Found 4469 items in 0.42 seconds

Processing Galatians at D:\TF\xml\09-galatians.xml

Found 2228 items in 0.20 seconds

Processing Ephesians at D:\TF\xml\10-ephesians.xml

Found 2419 items in 0.25 seconds

Processing Philippians at D:\TF\xml\11-philippians.xml

Found 1630 items in 0.15 seconds

Processing Colossians at D:\TF\xml\12-colossians.xml

Found 1575 items in 0.17 s

# 3 - Nestle1904LFT Text-Fabric production from pickle input<a class="anchor" id="bullet3"></a>
##### [Back to TOC](#TOC)

This script creates the Text-Fabric files by recursive calling the TF walker function.
API info: https://annotation.github.io/text-fabric/tf/convert/walker.html

The pickle files created by the script in  <a href="#bullet2x4">section 2.4</a> are stored on Github location [/resources/pickle](https://github.com/tonyjurg/Nestle1904LFT/tree/main/resources/pickle).

## 3.1 - Load libraries and initialize some data<a class="anchor" id="bullet3x1"></a>
##### [Back to TOC](#TOC)

The following global data initializes the Text-Fabric conversion script.

IMPORTANT: To ensure the proper creation of the Text-Fabric files on your system, it is crucial to adjust the values of BaseDir, PklDir, etc., to match the location of the data and the operating system you are using. This Jupyter Notebook employs the Windows operating system.

In [6]:
import pandas as pd
import os
import re
import gc
from tf.fabric import Fabric
from tf.convert.walker import CV
from tf.parameters import VERSION
from datetime import date
import pickle
from unidecode import unidecode
import unicodedata

BaseDir = 'D:\\TF\\'
XmlDir = BaseDir+'xml\\'
PklDir = BaseDir+'pkl\\'
XlsxDir = BaseDir+'xlsx\\'
CsvDir = BaseDir+'csv\\'


#         key: filename,       [0]=book_long,   [1]=book_num,  [3]=book_short
bo2book = {'01-matthew':       ['Matthew',         '1',        'Matt'],
           '02-mark':          ['Mark',            '2',        'Mark'],
           '03-luke':          ['Luke',            '3',        'Luke'],
           '04-john':          ['John',            '4',        'John'],
           '05-acts':          ['Acts',            '5',        'Acts'],
           '06-romans':        ['Romans',          '6',        'Rom'],
           '07-1corinthians':  ['I_Corinthians',   '7',        '1Cor'],
           '08-2corinthians':  ['II_Corinthians',  '8',        '2Cor'],
           '09-galatians':     ['Galatians',       '9',        'Gal'],
           '10-ephesians':     ['Ephesians',       '10',       'Eph'],
           '11-philippians':   ['Philippians',     '11',       'Phil'],
           '12-colossians':    ['Colossians',      '12',       'Col'],
           '13-1thessalonians':['I_Thessalonians', '13',       '1Thess'],
           '14-2thessalonians':['II_Thessalonians','14',       '2Thess'],
           '15-1timothy':      ['I_Timothy',       '15',       '1Tim'],
           '16-2timothy':      ['II_Timothy',      '16',       '2Tim'],
           '17-titus':         ['Titus',           '17',       'Titus'],
           '18-philemon':      ['Philemon',        '18',       'Phlm'],
           '19-hebrews':       ['Hebrews',         '19',       'Heb'],
           '20-james':         ['James',           '20',       'Jas'],
           '21-1peter':        ['I_Peter',         '21',       '1Pet'],
           '22-2peter':        ['II_Peter',        '22',       '2Pet'],
           '23-1john':         ['I_John',          '23',       '1John'],
           '24-2john':         ['II_John',         '24',       '2John'],
           '25-3john':         ['III_John',        '25',       '3John'],     
           '26-jude':          ['Jude',            '26',       'Jude'],
           '27-revelation':    ['Revelation',      '27',       'Rev']}

# unmark to debug using only one small book
#bo2book = {'18-philemon':      ['Philemon',        '18',       'Phlm']}

## 3.2 - Optionaly export to Excel for investigation<a class="anchor" id="bullet3x2"></a>
##### [Back to TOC](#TOC)

This step is optional. It will allow for manual examining the input data to the Text-Fabric conversion script.
Note that export to Excel format is very slow. Exporting to CSV is much faster.

In [ ]:
# Export to Excel file format
import openpyxl
import pickle

for bo in bo2book:
        '''
        load all data into a dataframe
        process books in order (bookinfo is a list!)
        '''   
        InputFile = os.path.join(PklDir, f'{bo}.pkl')
       
        print(f'\tloading {InputFile}...')
        pkl_file = open(InputFile, 'rb')
        df = pickle.load(pkl_file)
        pkl_file.close()
        df.to_excel(os.path.join(XlsxDir, f'{bo}.xlsx'), index=False)

	loading D:\TF\pkl\01-matthew.pkl...
	loading D:\TF\pkl\02-mark.pkl...
	loading D:\TF\pkl\03-luke.pkl...


In [56]:
# Export to CSV format 
import os
import pandas as pd
import pickle

for bo in bo2book:
    '''
    Load all data into a DataFrame
    Process books in order (bookinfo is a list!)
    '''   
    InputFile = os.path.join(PklDir, f'{bo}.pkl')
    OutputFile = os.path.join(CsvDir, f'{bo}.csv')
    
    print(f'\tloading {InputFile}...')
    
    # Using context manager for opening pickle file
    with open(InputFile, 'rb') as pkl_file:
        df = pickle.load(pkl_file)

    # Export DataFrame to CSV
    df.to_csv(OutputFile, index=False)
    print(f'\t{bo} data exported to {OutputFile}')


	loading D:\TF\pkl\01-matthew.pkl...
	01-matthew data exported to D:\TF\csv\01-matthew.csv
	loading D:\TF\pkl\02-mark.pkl...
	02-mark data exported to D:\TF\csv\02-mark.csv
	loading D:\TF\pkl\03-luke.pkl...
	03-luke data exported to D:\TF\csv\03-luke.csv
	loading D:\TF\pkl\04-john.pkl...
	04-john data exported to D:\TF\csv\04-john.csv
	loading D:\TF\pkl\05-acts.pkl...
	05-acts data exported to D:\TF\csv\05-acts.csv
	loading D:\TF\pkl\06-romans.pkl...
	06-romans data exported to D:\TF\csv\06-romans.csv
	loading D:\TF\pkl\07-1corinthians.pkl...
	07-1corinthians data exported to D:\TF\csv\07-1corinthians.csv
	loading D:\TF\pkl\08-2corinthians.pkl...
	08-2corinthians data exported to D:\TF\csv\08-2corinthians.csv
	loading D:\TF\pkl\09-galatians.pkl...
	09-galatians data exported to D:\TF\csv\09-galatians.csv
	loading D:\TF\pkl\10-ephesians.pkl...
	10-ephesians data exported to D:\TF\csv\10-ephesians.csv
	loading D:\TF\pkl\11-philippians.pkl...
	11-philippians data exported to D:\TF\csv\11-

## 3.3 - Running the TF walker function<a class="anchor" id="bullet3x3"></a>
##### [Back to TOC](#TOC)

API info: https://annotation.github.io/text-fabric/tf/convert/walker.html

Explanatory notes about the data interpretation logic are incorporated within the Python code of the director function.

In [8]:
from tf.fabric import Fabric
from tf.convert.walker import CV
TF = Fabric(locations=BaseDir, silent=False)
cv = CV(TF)

###############################################
#   Common helper functions                   #
###############################################

#Function to prevent errors during conversion due to missing data
def sanitize(input):
    if isinstance(input, float): return ''
    if isinstance(input, type(None)): return ''
    else: return (input)


# Function to expand the syntactic categories of words or wordgroup
# See also "MACULA Greek Treebank for the Nestle 1904 Greek New Testament.pdf" 
# page 5&6 (section 2.4 Syntactic Categories at Clause Level)
def ExpandRole(input):
    if input=="adv": return 'Adverbial'
    if input=="io":  return 'Indirect Object'
    if input=="o":   return 'Object'
    if input=="o2":  return 'Second Object'
    if input=="s":   return 'Subject'
    if input=="p":   return 'Predicate'
    if input=="v":   return 'Verbal'
    if input=="vc":  return 'Verbal Copula'
    if input=='aux': return 'Auxiliar'
    return ''

# Function to expantion of Part of Speech labels. See also the description in 
# "MACULA Greek Treebank for the Nestle 1904 Greek New Testament.pdf" page 6&7
# (2.2. Syntactic Categories at Word Level: Part of Speech Labels)
def ExpandSP(input):
    if input=='adj':  return 'Adjective'
    if input=='conj': return 'Conjunction'
    if input=='det':  return 'Determiner' 
    if input=='intj': return 'Interjection' 
    if input=='noun': return 'Noun' 
    if input=='num':  return 'Numeral' 
    if input=='prep': return 'Preposition' 
    if input=='ptcl': return 'Particle' 
    if input=='pron': return 'Pronoun' 
    if input=='verb': return 'Verb' 
    return ''


# Small function to remove accents from Greek words
def removeAccents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

###############################################
#          The director routine               #
###############################################

def director(cv):
    
    ###############################################
    #   Innitial setup of data etc.               #
    ###############################################
    NoneType = type(None)      # needed as tool to validate certain data
    IndexDict = {}             # init an empty dictionary
    WordGroupDict={}           # init a dummy dictionary
    PrevWordGroupSet = WordGroupSet = []
    PrevWordGroupList = WordGroupList = []
    RootWordGroup = 0
    WordNumber=FoundWords=WordGroupTrack=0
    # The following is required to recover succesfully from an abnormal condition
    # in the LowFat tree data where a <wg> element is labeled as <error>
    # this number is arbitrary but should be high enough not to clash with 'real' WG numbers
    DummyWGN=200000  # first dummy WG number
    
    # Following variables are used for textual critical data 
    criticalMarkCharacters = "[]()—"
    punctuationCharacters = ",.;·"
    translationTableMarkers = str.maketrans("", "", criticalMarkCharacters)
    translationTablePunctuations = str.maketrans("", "", punctuationCharacters)
    punctuations=('.',',',';','·')
    
    for bo,bookinfo in bo2book.items(): 
        
        ###############################################
        #   start of section executed for each book   #
        ###############################################
        
        # note: bookinfo is a list! Split the data
        Book        = bookinfo[0]  
        BookNumber  = int(bookinfo[1])
        BookShort   = bookinfo[2]
        BookLoc     = os.path.join(PklDir, f'{bo}.pkl') 
        
        # load  data for this book into a dataframe. 
        # make sure wordorder is correct
        print(f'\tWe are loading {BookLoc}...')
        pkl_file = open(BookLoc, 'rb')
        df_unsorted = pickle.load(pkl_file)
        pkl_file.close()
        
        '''
        Fill dictionary of column names for this book 
        sort to ensure proper wordorder
        '''
        ItemsInRow=1
        for itemname in df_unsorted.columns.to_list():
            IndexDict.update({'i_{}'.format(itemname): ItemsInRow})
            # This is to identify the collumn containing the key to sort upon
            if itemname=="{http://www.w3.org/XML/1998/namespace}id": SortKey=ItemsInRow-1
            ItemsInRow+=1
        
        df=df_unsorted.sort_values(by=df_unsorted.columns[SortKey])
        del df_unsorted

        # Set up nodes for new book
        ThisBookPointer = cv.node('book')
        cv.feature(ThisBookPointer, book=Book, booknumber=BookNumber, bookshort=BookShort)
        
        ThisChapterPointer = cv.node('chapter')
        cv.feature(ThisChapterPointer, chapter=1, book=Book)
        PreviousChapter=1
        
        ThisVersePointer = cv.node('verse')
        cv.feature(ThisVersePointer, verse=1, chapter=1, book=Book)
        PreviousVerse=1
        
        ThisSentencePointer = cv.node('sentence')
        cv.feature(ThisSentencePointer, sentence=1, headverse=1, chapter=1, book=Book)
        PreviousSentence=1

        ###############################################
        # Iterate through words and construct objects #
        ###############################################
        
        for row in df.itertuples():
            WordNumber += 1
            FoundWords +=1
                       
            # Detect and act upon changes in sentences, verse and chapter 
            # the order of terminating and creating the nodes is critical: 
            # close verse - close chapter - open chapter - open verse 
            NumberOfParents = sanitize(row[IndexDict.get("i_parents")])
            ThisSentence=int(row[IndexDict.get("i_Parent{}SN".format(NumberOfParents-1))])
            ThisVerse = sanitize(row[IndexDict.get("i_verse")])
            ThisChapter = sanitize(row[IndexDict.get("i_chapter")])
            
            if (ThisVerse!=PreviousVerse):
                cv.terminate(ThisVersePointer)
            
            if (ThisSentence!=PreviousSentence):
                cv.terminate(ThisSentencePointer)
                             
    
            if (ThisChapter!=PreviousChapter):
                cv.terminate(ThisChapterPointer)
                PreviousChapter = ThisChapter
                ThisChapterPointer = cv.node('chapter')
                cv.feature(ThisChapterPointer, chapter=ThisChapter, book=Book)
                
            if (ThisVerse!=PreviousVerse):
                PreviousVerse = ThisVerse  
                ThisVersePointer = cv.node('verse')
                cv.feature(ThisVersePointer, verse=ThisVerse, chapter=ThisChapter, book=Book)
                
            if (ThisSentence!=PreviousSentence):
                PreviousSentence=ThisSentence
                ThisSentencePointer = cv.node('sentence')
                cv.feature(ThisSentencePointer, sentence=ThisSentence, headverse=ThisVerse, chapter=ThisChapter, book=Book)       
        
            ###############################################
            #         analyze and process <WG> tags       #
            ###############################################
                    
            PrevWordGroupList=WordGroupList
            WordGroupList=[]  # stores current active WordGroup numbers
            
            for i in range(NumberOfParents-2,0,-1): # important: reversed itteration!
                
                _WGN=int(row[IndexDict.get("i_Parent{}WGN".format(i))])                             
                if _WGN!='':
                     WGN=int(_WGN)
                if WGN!='':
                     WGclass=sanitize(row[IndexDict.get("i_Parent{}Class".format(i))])
                     WGrule=sanitize(row[IndexDict.get("i_Parent{}Rule".format(i))])
                     WGtype=sanitize(row[IndexDict.get("i_Parent{}Type".format(i))])
                     if WGclass==WGrule==WGtype=='':
                         WGclass='empty'
                     else:
                         #print ('---',WordGroupList)
                         if WGN not in WordGroupList:
                             WordGroupList.append(WGN) 
                             #print(f'append WGN={WGN}')
                         WordGroupDict[(WGN,0)]=WGN
                         if WGrule[-2:]=='CL' and WGclass=='':  
                             WGclass='cl*'  # to simulate the way Logos presents this condition
                         WordGroupDict[(WGN,6)]=WGclass
                         WordGroupDict[(WGN,1)]=WGrule
                         WordGroupDict[(WGN,8)]=WGtype
                         WordGroupDict[(WGN,3)]=sanitize(row[IndexDict.get("i_Parent{}Junction".format(i))])
                         WordGroupDict[(WGN,2)]=sanitize(row[IndexDict.get("i_Parent{}Cltype".format(i))])
                         WordGroupDict[(WGN,7)]=sanitize(row[IndexDict.get("i_Parent{}Role".format(i))])
                         WordGroupDict[(WGN,9)]=sanitize(row[IndexDict.get("i_Parent{}Appos".format(i))])  # appos is not pressent any more in the newer dataset. kept here for the time being...
                         WordGroupDict[(WGN,10)]=NumberOfParents-1-i  # = number of parent wordgroups     
            if not PrevWordGroupList==WordGroupList:
                #print ('##',PrevWordGroupList,WordGroupList,NumberOfParents)
                if RootWordGroup != WordGroupList[0]:
                    RootWordGroup = WordGroupList[0]
                    SuspendableWordGoupList = []
                    # we have a new sentence. rebuild suspendable wordgroup list
                    # some cleaning of data may be added here to save on memmory... 
                    #for k in range(6): del WordGroupDict[item,k]
                for item in reversed(PrevWordGroupList):
                   if (item not in WordGroupList):
                         # CLOSE/SUSPEND CASE
                         SuspendableWordGoupList.append(item)
                         #print ('\n close: '+str(WordGroupDict[(item,0)])+' '+ WordGroupDict[(item,6)]+' '+ WordGroupDict[(item,1)]+' '+WordGroupDict[(item,8)],end=' ')                     
                         cv.terminate(WordGroupDict[(item,4)])
                for item in WordGroupList:
                    if (item not in PrevWordGroupList):
                         if (item in SuspendableWordGoupList):
                              # RESUME CASE
                              #print ('\n resume: '+str(WordGroupDict[(item,0)])+' '+ WordGroupDict[(item,6)]+' '+WordGroupDict[(item,1)]+' '+WordGroupDict[(item,8)],end=' ') 
                              cv.resume(WordGroupDict[(item,4)])
                         else:
                              # CREATE CASE
                              #print ('\n create: '+str(WordGroupDict[(item,0)])+' '+ WordGroupDict[(item,6)]+' '+ WordGroupDict[(item,1)]+' '+WordGroupDict[(item,8)],end=' ')
                              WordGroupDict[(item,4)]=cv.node('wg')
                              WordGroupDict[(item,5)]=WordGroupTrack
                              WordGroupTrack += 1
                              cv.feature(WordGroupDict[(item,4)], wgnum=WordGroupDict[(item,0)], junction=WordGroupDict[(item,3)], 
                                         clausetype=WordGroupDict[(item,2)], wgrule=WordGroupDict[(item,1)], wgclass=WordGroupDict[(item,6)], 
                                         wgrole=WordGroupDict[(item,7)],wgrolelong=ExpandRole(WordGroupDict[(item,7)]),
                                         wgtype=WordGroupDict[(item,8)],wglevel=WordGroupDict[(item,10)])
     
            # These roles are performed either by a WG or just a single word.
            Role=row[IndexDict.get("i_role")]
            ValidRoles=["adv","io","o","o2","s","p","v","vc","aux"]
            DistanceToRoleClause=0
            if isinstance (Role,str) and Role in ValidRoles: 
                # Role is assign to this word (uniqely)
                WordRole=Role
                WordRoleLong=ExpandRole(WordRole)
            else:
                # Role details needs to be taken from some uptree wordgroup 
                WordRole=WordRoleLong=''
                for item in range(1,NumberOfParents-1):
                    Role = sanitize(row[IndexDict.get("i_Parent{}Role".format(item))])
                    if isinstance (Role,str) and Role in ValidRoles: 
                        WordRole=Role        
                        WordRoleLong=ExpandRole(WordRole)
                        DistanceToRoleClause=item
                        break
                        
            # Find the number of the WG containing the clause definition
            for item in range(1,NumberOfParents-1):
                WGrule = sanitize(row[IndexDict.get("i_Parent{}Rule".format(item))])
                if row[IndexDict.get("i_Parent{}Class".format(item))]=='cl' or WGrule[-2:]=='CL':  
                    ContainedClause=sanitize(row[IndexDict.get("i_Parent{}WGN".format(item))])
                    break

            ###############################################
            #         analyze and process <W> tags        #
            ###############################################
                
            # Determine syntactic categories at word level. 
            PartOfSpeech=sanitize(row[IndexDict.get("i_class")])
            PartOfSpeechFull=ExpandSP(PartOfSpeech)
            
            # The folling part of code reproduces feature 'word' and 'after' that are
            # currently containing incorrect data in a few specific cases.
            # See https://github.com/tonyjurg/Nestle1904LFT/blob/main/resources/identifying_odd_afters.ipynb
            # Get the word details and detect presence of punctuations
            # it also creates the textual critical features

            rawWord=sanitize(row[IndexDict.get("i_unicode")])
            cleanWord= rawWord.translate(translationTableMarkers)
            rawWithoutPunctuations=rawWord.translate(translationTablePunctuations)
            markBefore=markAfter=PunctuationMarkOrder=''
            if cleanWord[-1] in punctuations:
                punctuation=cleanWord[-1]
                after=punctuation+' '
                word=cleanWord[:-1]
            else:
                after=' '
                word=cleanWord
                punctuation=''
            if rawWithoutPunctuations!=word:
                markAfter=markBefore=''
                if rawWord.find(word)==0:
                    markAfter=rawWithoutPunctuations.replace(word,"")
                    if punctuation!='':
                        if rawWord.find(markAfter)-rawWord.find(punctuation)>0:
                            PunctuationMarkOrder="3" # punct. before mark
                        else:
                            PunctuationMarkOrder="2" # punct. after mark.
                    else:
                        PunctuationMarkOrder="1" #no punctuation, mark after word
                else:
                    markBefore=rawWithoutPunctuations.replace(word,"")
                    PunctuationMarkOrder="0" #mark is before word
                    
            # Some attributes are not present inside some (small) books. The following is to prevent exceptions.
            degree='' 
            if 'i_degree'  in IndexDict: 
                degree=sanitize(row[IndexDict.get("i_degree")]) 
            subjref=''
            if 'i_subjref' in IndexDict: 
                subjref=sanitize(row[IndexDict.get("i_subjref")]) 

                    
            # Create the word slots
            this_word = cv.slot()
            cv.feature(this_word, 
                    after=         after,
                    unicode=       rawWord,
                    word=          word,
                    wordtranslit=  unidecode(word),
                    wordunacc=     removeAccents(word),
                    punctuation=   punctuation,
                    markafter=     markAfter,
                    markbefore=    markBefore,
                    markorder=     PunctuationMarkOrder,
                    monad=         FoundWords,
                    orig_order=    sanitize(row[IndexDict.get("i_word_order")]),
                    book=          Book,
                    booknumber=    BookNumber,
                    bookshort=     BookShort,
                    chapter=       ThisChapter,
                    ref=           sanitize(row[IndexDict.get("i_ref")]),
                    sp=            PartOfSpeech,
                    sp_full=       PartOfSpeechFull,
                    verse=         ThisVerse,
                    sentence=      ThisSentence,
                    normalized=    sanitize(row[IndexDict.get("i_normalized")]),
                    morph=         sanitize(row[IndexDict.get("i_morph")]),
                    strongs=       sanitize(row[IndexDict.get("i_strong")]),
                    lex_dom=       sanitize(row[IndexDict.get("i_domain")]),
                    ln=            sanitize(row[IndexDict.get("i_ln")]),
                    gloss=         sanitize(row[IndexDict.get("i_gloss")]),
                    gn=            sanitize(row[IndexDict.get("i_gender")]),
                    nu=            sanitize(row[IndexDict.get("i_number")]),
                    case=          sanitize(row[IndexDict.get("i_case")]),
                    lemma=         sanitize(row[IndexDict.get("i_lemma")]),
                    person=        sanitize(row[IndexDict.get("i_person")]),
                    mood=          sanitize(row[IndexDict.get("i_mood")]),
                    tense=         sanitize(row[IndexDict.get("i_tense")]),
                    number=        sanitize(row[IndexDict.get("i_number")]),
                    voice=         sanitize(row[IndexDict.get("i_voice")]),
                    degree=        degree,
                    type=          sanitize(row[IndexDict.get("i_type")]),
                    reference=     sanitize(row[IndexDict.get("i_ref")]),     
                    subj_ref=      subjref,
                    nodeID=        sanitize(row[4]),         #this is a fixed position in dataframe
                    wordrole=      WordRole,
                    wordrolelong=  WordRoleLong,
                    wordlevel=     NumberOfParents-1,
                    roleclausedistance = DistanceToRoleClause,
                    containedclause = ContainedClause
                    )
            cv.terminate(this_word)   
   
        
        '''
        wrap up the book. At the end of the book we need to close all nodes in proper order.
        '''   
        # close all open WordGroup nodes
        for item in WordGroupList:
            #cv.feature(WordGroupDict[(item,4)], add some stats?)
            cv.terminate(WordGroupDict[item,4])

        cv.terminate(ThisSentencePointer)
        cv.terminate(ThisVersePointer)
        cv.terminate(ThisChapterPointer)      
        cv.terminate(ThisBookPointer)

        # clear dataframe for this book, clear the index dictionary
        del df
        IndexDict.clear()
        gc.collect()
        
        ###############################################
        #    end of section executed for each book    #
        ###############################################

    ###############################################
    #      end of director function               #
    ###############################################
        
###############################################
#            Output definitions               #
###############################################
        
slotType = 'word'  
otext = {  # dictionary of config data for sections and text formats
        'fmt:text-orig-full':     '{word}{after}',
        'fmt:text-normalized':    '{normalized}{after}',
        'fmt:text-unaccented':    '{wordunacc}{after}',
        'fmt:text-transliterated':'{wordtranslit}{after}', 
        'fmt:text-critical':      '{unicode} ',
        'sectionTypes':'book,chapter,verse',
        'sectionFeatures':'book,chapter,verse',
        'structureFeatures': 'book,chapter,verse',
        'structureTypes': 'book,chapter,verse',
        }

# configure metadata
generic = {  # dictionary of metadata meant for all features
        'textFabriVersion': '{}'.format(VERSION),  #imported from tf.parameter
        'xmlSourceLocation': 'https://github.com/tonyjurg/Nestle1904LFT/tree/main/resources/xml/20240210',
        'xmlSourceDate': 'February 10, 2024',
        'author': 'Evangelists and apostles',
        'availability': 'Creative Commons Attribution 4.0 International (CC BY 4.0)',
        'converters': 'Tony Jurg',
        'converterSource': 'https://github.com/tonyjurg/Nestle1904LFT/tree/main/resources/converter',
        'converterVersion': '0.7',
        'dataSource': 'MACULA Greek Linguistic Datasets, available at https://github.com/Clear-Bible/macula-greek/tree/main/Nestle1904/nodes',
        'editors': 'Eberhart Nestle (1904)',
        'sourceDescription': 'Greek New Testment (British Foreign Bible Society, 1904)',
        'sourceFormat': 'XML (Low Fat tree XML data)',
        'title': 'Greek New Testament (Nestle1904LFT)'
         }

# set of integer valued feature names
intFeatures = {  
         'booknumber',
         'chapter',
         'verse',
         'sentence',
         'wgnum',
         'orig_order',
         'monad',
         'wglevel'
         }

# per feature dicts with metadata
# icon provides guidance on feature maturity (✅ = trustworthy, 🆗  = usable, ⚠️ = be carefull when using)
# 
featureMeta = {  
                 'after':       {'description': '✅ Characters (eg. punctuations) following the word'},
                 'book':        {'description': '✅ Book name (in English language)'},
                 'booknumber':  {'description': '✅ NT book number (Matthew=1, Mark=2, ..., Revelation=27)'},
                 'bookshort':   {'description': '✅ Book name (abbreviated)'},
                 'chapter':     {'description': '✅ Chapter number inside book'},
                 'verse':       {'description': '✅ Verse number inside chapter'},
                 'headverse':   {'description': '✅ Start verse number of a sentence'},
                 'sentence':    {'description': '✅ Sentence number (counted per chapter)'},
                 'type':        {'description': '✅ Wordgroup type information (e.g.verb, verbless, elided, minor)'},
                 'wgrule':      {'description': '✅ Wordgroup rule information (e.g. Np-Appos, ClCl2, PrepNp)'},
                 'orig_order':  {'description': '✅ Word order (in source XML file)'},
                 'monad':       {'description': '✅ Monad (smallest token matching word order in the corpus)'},
                 'word':        {'description': '✅ Word as it appears in the text (excl. punctuations)'},
                 'wordtranslit':{'description': '🆗 Transliteration of the text (in latin letters, excl. punctuations)'},
                 'wordunacc':   {'description': '✅ Word without accents (excl. punctuations)'},
                 'unicode':     {'description': '✅ Word as it apears in the text in Unicode (incl. punctuations)'},
                 'punctuation': {'description': '✅ Punctuation after word'},
                 'markafter':   {'description': '🆗 Text critical marker after word'},
                 'markbefore':  {'description': '🆗 Text critical marker before word'},
                 'markorder':   {'description': ' Order of punctuation and text critical marker'},
                 'ref':         {'description': '✅ Value of the ref ID (taken from XML sourcedata)'},
                 'sp':          {'description': '✅ Part of Speech (abbreviated)'},
                 'sp_full':     {'description': '✅ Part of Speech (long description)'}, 
                 'normalized':  {'description': '✅ Surface word with accents normalized and trailing punctuations removed'},
                 'lemma':       {'description': '✅ Lexeme (lemma)'},
                 'morph':       {'description': '✅ Morphological tag (Sandborg-Petersen morphology)'},
                                 # see also discussion on relation between lex_dom and ln 
                                 # @ https://github.com/Clear-Bible/macula-greek/issues/29
                 'lex_dom':     {'description': '✅ Lexical domain according to Semantic Dictionary of Biblical Greek, SDBG (not present everywhere?)'},
                 'ln':          {'description': '✅ Lauw-Nida lexical classification (not present everywhere?)'},
                 'strongs':     {'description': '✅ Strongs number'},
                 'gloss':       {'description': '✅ English gloss'},
                 'gn':          {'description': '✅ Gramatical gender (Masculine, Feminine, Neuter)'},
                 'nu':          {'description': '✅ Gramatical number (Singular, Plural)'},
                 'case':        {'description': '✅ Gramatical case (Nominative, Genitive, Dative, Accusative, Vocative)'},
                 'person':      {'description': '✅ Gramatical person of the verb (first, second, third)'},
                 'mood':        {'description': '✅ Gramatical mood of the verb (passive, etc)'},
                 'tense':       {'description': '✅ Gramatical tense of the verb (e.g. Present, Aorist)'},
                 'number':      {'description': '✅ Gramatical number of the verb (e.g. singular, plural)'},
                 'voice':       {'description': '✅ Gramatical voice of the verb (e.g. active,passive)'},
                 'degree':      {'description': '✅ Degree (e.g. Comparitative, Superlative)'},
                 'type':        {'description': '✅ Gramatical type  of noun or pronoun (e.g. Common, Personal)'},
                 'reference':   {'description': '✅ Reference (to nodeID in XML source data, not yet post-processes)'},
                 'subj_ref':    {'description': '🆗 Subject reference (to nodeID in XML source data, not yet post-processes)'},
                 'nodeID':      {'description': '✅ Node ID (as in the XML source data)'},
                 'junction':    {'description': '✅ Junction data related to a wordgroup'},
                 'wgnum':       {'description': '✅ Wordgroup number (counted per book)'},
                 'wgclass':     {'description': '✅ Class of the wordgroup (e.g. cl, np, vp)'},
                 'wgrole':      {'description': '✅ Syntactical role of the wordgroup (abbreviated)'},
                 'wgrolelong':  {'description': '✅ Syntactical role of the wordgroup (full)'},
                 'wordrole':    {'description': '✅ Syntactical role of the word (abbreviated)'},
                 'wordrolelong':{'description': '✅ Syntactical role of the word (full)'},
                 'wgtype':      {'description': '✅ Wordgroup type details (e.g. group, apposition)'},
                 'clausetype':  {'description': '✅ Clause type details (e.g. Verbless, Minor)'},
                 'wglevel':     {'description': '🆗 Number of the parent wordgroups for a wordgroup'},
                 'wordlevel':   {'description': '🆗 Number of the parent wordgroups for a word'},
                 'roleclausedistance': {'description': '⚠️ Distance to the wordgroup defining the syntactical role of this word'},
                 'containedclause': {'description': '🆗 Contained clause (WG number)'}
                 }


###############################################
#            the main function                #
###############################################

good = cv.walk(
    director,
    slotType,
    otext=otext,
    generic=generic,
    intFeatures=intFeatures,
    featureMeta=featureMeta,
    warn=True,
    force=True
)

if good:
  print ("done")

This is Text-Fabric 12.2.2
0 features found and 0 ignored
  0.00s Not all of the warp features otype and oslots are present in
D:/TF
  0.00s Only the Feature and Edge APIs will be enabled
  0.00s Warp feature "otext" not found. Working without Text-API

  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |   SECTION   TYPES:    book, chapter, verse
   |   SECTION   FEATURES: book, chapter, verse
   |   STRUCTURE TYPES:    book, chapter, verse
   |   STRUCTURE FEATURES: book, chapter, verse
   |   TEXT      FEATURES:
   |      |   text-critical        unicode
   |      |   text-normalized      after, normalized
   |      |   text-orig-full       after, word
   |      |   text-transliterated  after, wordtranslit
   |      |   text-unaccented      after, wordunacc
   |     0.00s OK
   |     0.00s Following director... 
	We are loading D:\TF\pkl\01-matthew.pkl...
	We are loading D:\TF\pkl\02-mark.pkl...
	We are loading D:\TF\pkl\03-luke.pkl..

# 4 -  Publish product on github<a class="anchor" id="bullet4"></a>
##### [Back to TOC](#TOC)

TBD

In [1]:
# Load TF code

from tf.fabric import Fabric
from tf.app import use

In [4]:
# Define the repository
ORG = "tonyjurg"
REPO = "Nestle1904LFT"

# Added details for the release
MESSAGE = "New release"
DESCRIPTION = """
This release uses a new dataset. 

The main difference is in feature Strongs:
  * Some errors were corrected
  * composite words are now with two or more Strong values
"""

In [7]:
# load the app and data
N1904 = use ("tonyjurg/Nestle1904LFT", version="0.7", hoist=globals())

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,27,5102.93,100
chapter,260,529.92,100
verse,7943,17.35,100
sentence,8011,17.20,100
wg,105430,6.85,524
word,137779,1.00,100
